In [1]:
from utils import *
from opennmt.utils.misc import print_bytes

In [2]:
def session_init_op(_scaffold, sess):
    saver.restore(sess, checkpoint_path)
    tf.logging.info("Restored model from %s", checkpoint_path)

In [3]:
model_dir = './trained_model'

src = './data/unsupervised-nmt-enfr/newstest2014.en.tok'
tgt = './data/unsupervised-nmt-enfr/newstest2014.fr.tok'
src_vocab = './data/unsupervised-nmt-enfr/en-vocab.txt'
tgt_vocab = './data/unsupervised-nmt-enfr/fr-vocab.txt'
direction = 1

In [4]:
if direction == 1:
    src_file, tgt_file = src, tgt
    src_vocab_file, tgt_vocab_file = src_vocab, tgt_vocab
else:
    src_file, tgt_file = tgt, src
    src_vocab_file, tgt_vocab_file = tgt_vocab, src_vocab

In [5]:
tgt_vocab_size = count_lines(tgt_vocab_file) + 1
src_vocab_size = count_lines(src_vocab_file) + 1
src_vocab = tf.contrib.lookup.index_table_from_file(src_vocab_file, vocab_size=src_vocab_size - 1, num_oov_buckets=1)

with tf.device("cpu:0"):
    src_iterator = load_data(src_file, src_vocab)

src = src_iterator.get_next()

In [6]:
hidden_size = 512
encoder = onmt.encoders.BidirectionalRNNEncoder(2, hidden_size)
decoder = onmt.decoders.AttentionalRNNDecoder(2, hidden_size, bridge=onmt.layers.CopyBridge())

with tf.variable_scope("src" if direction == 1 else "tgt"):
    src_emb = tf.get_variable("embedding", shape=[src_vocab_size, 300])
    src_gen = tf.layers.Dense(src_vocab_size)
    src_gen.build([None, hidden_size])

with tf.variable_scope("tgt" if direction == 1 else "src"):
    tgt_emb = tf.get_variable("embedding", shape=[tgt_vocab_size, 300])
    tgt_gen = tf.layers.Dense(tgt_vocab_size)
    tgt_gen.build([None, hidden_size])

In [7]:
encoder_output = encode(encoder, src_emb, src)
sampled_ids, sampled_length = decode(encoder_output, decoder, tgt_emb, tgt_vocab_size, tgt_gen, src)

In [8]:
tgt_vocab_rev = tf.contrib.lookup.index_to_string_table_from_file(tgt_vocab_file, vocab_size=tgt_vocab_size - 1,\
                                                                  default_value=constants.UNKNOWN_TOKEN)

In [9]:
tokens = tgt_vocab_rev.lookup(tf.cast(sampled_ids, tf.int64))
length = sampled_length

In [10]:
saver = tf.train.Saver()
checkpoint_path = tf.train.latest_checkpoint(model_dir)

In [11]:
checkpoint_path

'./trained_model\\model.ckpt-7576'

In [13]:
scaffold = tf.train.Scaffold(init_fn=session_init_op)
session_creator = tf.train.ChiefSessionCreator(scaffold=scaffold)

In [13]:
with tf.train.MonitoredSession(session_creator=session_creator) as sess:
    sess.run(src_iterator.initializer)
    while not sess.should_stop():
        _tokens, _length = sess.run([tokens, length])
        for b in range(_tokens.shape[0]):
            pred_toks = _tokens[b][0][:_length[b][0] - 1]
            pred_sent = b" ".join(pred_toks)
            print_bytes(pred_sent)

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./trained_model\model.ckpt-7576
INFO:tensorflow:Restored model from ./trained_model\model.ckpt-7576
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
ce est une absence .
le canada est le transport de la guerre .
vous vous aller !
cette commentaires ?


In [ ]:
with tf.train.MonitoredSession(session_creator=session_creator) as sess:
    sess.run(src_iterator.initializer)
    while not sess.should_stop():
        _tokens, _length = sess.run([tokens, length])
        for b in range(_tokens.shape[0]):
            pred_toks = _tokens[b][0][:_length[b][0] - 1]
            pred_sent = b" ".join(pred_toks)
            print_bytes(pred_sent)

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./trained_model\model.ckpt-7576
INFO:tensorflow:Restored model from ./trained_model\model.ckpt-7576
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
<unk> <unk> <unk> quelques années .
<unk> <unk> <unk> <unk> <unk> à un navire , <unk> , la construction de la république , en mercredi .
<unk> <unk> , <unk> , <unk> ont eu pendant la fête du nord <unk> à 10 h h . à l ' ouest est situé à la ville de environ 000 <unk> et les soldats ont été réalisé à <unk> .
une hôtel , dans votre langue !
comme le canada , l ’ air et face à l ’ importation à une force <unk> - <unk> , les pays sont en <unk> à une solution en plus une nouvelle <unk> , ce <unk> , cette méthode du client .
le matériel d ’ impôt , une journée <unk> <unk> et <unk> et vérifier les renseignements , ils sont <unk> , dans l ' établissement de une approche positive en <unk> , la société et des services de la système interna

permettez - professeur , professeur , chef ministre , ministre , je suis heureux : m . je s ' est à faire à la presse , nous nous avons trouver à faire faire cette solution .
entre la coopération , les ministres , ont été eu une phase des plaintes que l ' université du président m . son m .
<unk> , <unk> , la chambre des ministres , le comité fédéral , que cela , il a été fait à faire une attention à m . <unk> comme un juge » sans faire une position sans <unk> , <unk> et les relations , il doit être été appliquées .
m . <unk> , une demande ( le ministre , le ministre , publié une version de façon avec m . <unk> sur le menu .
elle était ensuite faite sur votre visite au siège par les affaires .
m . dit que que , par l ' expert du tribunal du canada , m . <unk> ne ne ne être pas à répondre à la famille , il lui <unk> .
si m . <unk> par la france , il peut assurer que qu ' elle puisse être pas à « « « <unk> .
m . <unk> dans son lettre , elle avait pu une véritable grave pour un climat eff

même tout m .
m . <unk> <unk> est une grande grande famille et s ' est à plus plus de l ' utilisation des emplois .
j ' m . j ' ai pas pas jamais .
il sont plus plus des niveaux de comptabilité d ' emploi en <unk> .
ne ne pas pas ! !
deux deux deux heures au moment de plus dans les pays pays sont deux deux heures pendant le président .
en france , les pays d ' autres pays , plus lieu au cours de deux heures .
je ne <unk> m . t ´ devons mes <unk> .
je j ' est un petit comme le tourisme , les produits , <unk> , <unk> , une entreprise d ’ entreprise de cuisine de vie .
pourquoi ne pourquoi le client chose bien ?
t ’ t t t faut accès à long .
mais , si la valeur est la famille et la famille sont être comme , c ’ elle est .
comment une solution de la procédure ne vous <unk> entre la <unk> .
si vous avait reçu le service d ’ une marque avant après le début que l ' affaire , je , j ' h % de l ’ employés ont été pas .
mais , m . m ’ est que le restaurant <unk> est un <unk> .
après le point , ,

je ne dit que l ' heure .
je a eu une véritable relation avec lui lui avait un mère et art .
après le point , il était pas trop de m et je s ' avait à quitter .
après le temps , <unk> .
pour un plus temps , le même <unk> , je vous <unk> , je , je <unk> , j ' ai dit , j ' <unk> .
elle avait ouverte à m , nous devons <unk> , <unk> , mais le président .
je me voudrais à m . je nous voté en <unk> , <unk> , lui lui lui appel .
mais ce n ' <unk> .
je ne existe un plus bon vie .
je lui me me me .
il avait eu un plus plus tard , après le même <unk> .
dans la fin , il va souvent sûr et faire à faire , conséquent , conséquent .
je suis t t veulent qu ’ il a dit que les gens sont pas pas trop trop .
je sont pas plus plus plus tard je nous faire à eux à mon .
en plus en temps , tout tout de la famille , je me veulent de leurs collègues , je , j ' ai dit à tous vos questions ne ne ne <unk> , nous ne ne pas pas ?
j ' aimerais de leurs hommes , mais pas tout tard , mais deux deux plus tard pour le te

sans modification à l ' argent était <unk> , <unk> , le nom de « « « " » se réfère à l ’ accueil sur les mars , le fait des effets de la maladie a été <unk> .
elle avait dit le « <unk> à votre <unk> .
on offre une valeur de signalisation
avec les chambres , <unk> , 3 . 3 minutes de <unk> <unk> en <unk> de la guerre , la loi pour la <unk> <unk> et <unk> et vous permet un <unk> un <unk> .
« le canada , plus , dans les régions , la loi est été été par les biens ou par plusieurs entreprises .
l ' ouest a commencé à obtenir les coûts à revenus .
il a a été un long dans la gestion de l ’ entreprise de service et <unk> .
new york fut situé pendant pied .
il est une augmentation en <unk> avec new international .
un un <unk> <unk> au 30 h . les pays .
au moins de deux personnes ont été appliquées , <unk> à police .
il est un service de travail pour l ' organisation économique ( sécurité ) ) ) , d ' autres d ' <unk> .
la tendance était en <unk> en ligne , une <unk> de <unk> .
les installations d

...

l ' <unk> <unk> fut alors une tendance une attaque <unk> à un <unk> <unk> par le navire fut situé pendant 10 heures sur 10 mars 2008 .
m . dit dans l ’ air <unk> , une fois un <unk> avait un navire <unk> avec <unk> avec une heures .
elle avait eu une période - longue de <unk> dans la guerre de la guerre , <unk> , <unk> dans les deux jours dans le milieu .
m . dit <unk> <unk> a été besoin que la façon de deux autres autres membres de l ' <unk> <unk> <unk> <unk> pour l ' échelle <unk> - <unk> , <unk> et <unk> - <unk> ont été deux deux années .
cela , ce c ' souhaite pas la violation de vote .
à quitter à m , bien , , le t n ' est pas à quitter .
à aller à 10 ans , <unk> , <unk> et <unk> et les villes , qu ’ elle est pas .
<unk> s ' est lieu de l ' université de quitter .
elle avait été interprétés pour une hausse au niveau en années qui était légèrement dans la année en 2004 , la région de la ville <unk> , <unk> , <unk> , <unk> <unk> <unk> <unk> ou deux deux mois mois .
pendant la fin , 

<unk> ne ils ont avoir un fichier que on <unk> à l ’ hôtel <unk> - <unk> .
mais tout avec les gens plus plus plus un plus - plus au point à faire une attention du client du canada .
le président s ' veulent à trouver les soldats depuis le musée des solutions complexes et <unk> , <unk> et leurs hommes .
vie de la salle ( vice - président
un vice - président de la région de la région , en <unk> de la république , de la région , à moins au niveau international sur genève .
elle avait été tués à l ' arrêt en vigueur en 2000 , <unk> , fait de jour à 10 000 $ , , par les bureaux et bureaux dans la région .
entre entre 2001 , 2001 , le président - président , qui a déclaré le président de l ' institut de <unk> , un service public , ont entreprises entreprises , les services , les transactions , en base de base ou les transactions , le système de la convention en sierra .
monsieur le président , <unk> , <unk> - 2008 a été fait la lutte contre un terrorisme international - processus , que qu ' 

une examen comprenait que les <unk> <unk> <unk> du ministre du canada a été réalisé par les commentaires suivantes sont <unk> dans le cours de année en cours en cours de juin 2008 .
« une nouvelle vision du ministère du ministère , <unk> , « le concept est une déclaration .
il a avait décidé à mettre une procédure ou une procédure international pour mme <unk> par le greffier " du client » .
il avait également <unk> <unk> <unk> sur le groupe de travail et fournir une demande de référence de <unk> , <unk> , s ' appliquer que l ' appui des personnes au poste et <unk> et présenter sa lettre .
mme m . avec les articles pour la demande de appel des plaintes , une licence - service de <unk> <unk> .
après le lendemain avait été m , après le poste - <unk> - <unk> concernant la chambre <unk> , je lui lui un véritable véritable propre .
mais m . dit par le système de guerre , elle avait déposé sur les images .
dans ce sens , j ' ai <unk> aux dommages à <unk> et s ' il offre à la santé publique du

les chercheurs ont été plus sur sur les effets de l ' air du navire du navire <unk> sont été à sa position .
cette étude ont été réalisé que les <unk> <unk> <unk> à la famille ( <unk> à l ' <unk> du monde , le point de vue d ' <unk> , chez les <unk> <unk> .
mais beaucoup , nous avons parlé que les <unk> <unk> et <unk> à répondre des espèces .
mme <unk> <unk> , <unk> <unk> <unk> à la région de presse , comme : c ’ est très très très dans la plupart <unk> , la famille et la famille de la famille de la famille sont très en <unk> . </s>
nous avons commencé à faire à cette fois , les espèces .
on a ajouté un plus très dans les espèces , le droit de la famille de la <unk> est responsable pour <unk> - <unk> et <unk> et la famille - <unk> et la chine ont été très des progrès culturelles .
à plus plus plus plus plus trouver les valeurs <unk> à l ' <unk> - <unk> <unk> de <unk> , les produits , l ’ association ont souvent <unk> et les salles de tissus .
ils étaient pas les eaux du monde dans les 

l ' office du tribunal a permis au jour des mois dans l ' assemblée , qui peut être mis en même si l ’ eau et les produits sont <unk> .
mais m . on n ' était pas des fonctionnaires qui ont pu une situation de vie et <unk> , m . m ' avait pas à participer aux programmes .
« m ’ a un une dimension de la dimension , il est à droit au droit ou pas pas pas pas pas à la personne ou pas pas pas pas à une façon d ' avoir pas pas ou pas ou <unk> , m .
pendant deux deux mois de la cour , la commission a dit m . le nom de la fête de la ville <unk> <unk> par 15 voix , après une marque .
m . <unk> pour un système de procédure sur la durée de jour .
m . m . m et <unk> et <unk> .
ce document ne été soumis pendant deux mois pendant les voix !
« nous devons besoin à une capacité efficace » » » dit .
il est pourquoi que nous veulent <unk> t t a un un système d ’ ils <unk> en <unk> , votre ligne , sur une entreprise , avoir avoir un plus plus plus à 10 h .
m . dit que l ' utilisation de signalisation à r